### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [418]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error 

In [419]:
import numpy as np
import matplotlib.pyplot as plt

In [420]:
cali = pd.read_csv('data/housing.csv')

In [421]:
cali.head(15)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [422]:
cali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


##### Step 1 : Clean Dataset and fill in gaps as required per analysis

In [423]:
cali.query('total_bedrooms.isna()')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18.0,3620.0,NaN,3171.0,779.0,3.3409,220500.0,NEAR OCEAN
20268,-119.18,34.19,19.0,2393.0,NaN,1938.0,762.0,1.6953,167400.0,NEAR OCEAN
20372,-118.88,34.17,15.0,4260.0,NaN,1701.0,669.0,5.1033,410700.0,<1H OCEAN
20460,-118.75,34.29,17.0,5512.0,NaN,2734.0,814.0,6.6073,258100.0,<1H OCEAN


In [424]:
#Drop NA for the total_bedrooms column given the low % of NA values for the column at ~ some percentage 

cali_df = cali.dropna()

In [425]:
cali_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


##### Step 1: Review unique values sampling for feature set 

In [426]:
col_features = cali_df.columns.values.tolist() 
for item in col_features: 
    print('{0} \t {1} \n'.format(item, cali_df[item].unique()))

longitude 	 [-122.23 -122.22 -122.24 -122.25 -122.26 -122.27 -122.28 -122.29 -122.3
 -122.21 -122.2  -122.19 -122.18 -122.13 -122.16 -122.17 -122.15 -122.14
 -122.12 -122.33 -122.34 -122.06 -122.07 -122.08 -122.09 -122.1  -122.11
 -122.03 -121.97 -122.02 -122.04 -122.05 -121.99 -122.01 -121.96 -121.98
 -122.   -121.93 -121.94 -121.95 -121.92 -121.89 -121.91 -121.9  -121.88
 -121.87 -121.85 -121.86 -121.84 -121.82 -121.77 -121.62 -121.61 -121.72
 -121.73 -121.75 -121.8  -121.76 -121.78 -121.79 -119.78 -119.93 -120.
 -120.56 -120.59 -120.55 -120.25 -120.79 -120.8  -120.65 -120.76 -120.88
 -120.69 -120.93 -120.97 -120.87 -120.98 -120.72 -120.77 -120.66 -120.62
 -120.71 -121.83 -121.81 -121.74 -121.68 -121.54 -121.51 -121.59 -121.58
 -121.6  -121.63 -121.57 -121.65 -121.64 -121.71 -121.66 -121.56 -121.5
 -121.41 -121.39 -121.24 -121.19 -121.36 -121.46 -121.49 -121.44 -121.47
 -121.53 -121.52 -121.55 -121.67 -121.69 -121.7  -120.46 -120.54 -120.67
 -120.9  -120.91 -120.57 -120.43 -120.42 -1

##### Step 2: Create the Train and Test split to establish the dependent variable as well

In [427]:
X = cali_df.drop('median_house_value', axis = 1)
y = cali_df['median_house_value']

In [428]:
#Convention to establish variables for Training and Testing sets splits
X_train, X_test, y_train, y_test = '', '', '', ''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=38)

#check resulting traing and test data frames
print(X_train.shape)
print(X_test.shape)
print(type(X_train), type(y_train))

(16346, 9)
(4087, 9)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


##### Step 3: Run the baseline model performance exercise to have a basis for comparing all other models tried (Following Baseline Perdictions method 8.7)

In [429]:
#Baseline Prediction Method from 8.7 to establish MSE for train and test data sets: 

baseline_train = ''
baseline_test = ''
mse_baseline_train = ''
mse_baseline_test = ''

baseline_train = np.ones(shape = y_train.shape)*y_train.mean()
baseline_test = np.ones(shape = y_test.shape)*y_test.mean()

mse_baseline_train = mean_squared_error(baseline_train, y_train)
mse_baseline_test = mean_squared_error(baseline_test, y_test)

print(baseline_train.shape, baseline_test.shape)
print(f'Baseline for training data: {mse_baseline_train}')
print(f'Baseline for testing data: {mse_baseline_test}')


(16346,) (4087,)
Baseline for training data: 13292052915.12833
Baseline for testing data: 13454773488.761948


##### Step 4: Examine correlations (ensure this is after creating your train / test split data sets)

In [430]:
highest_corr = ''

#highest_corr = train.corr()[['SalePrice']].nlargest(columns = 'SalePrice', n = 2).index[1]
highest_corr=cali_df.corr(numeric_only=True)[['median_house_value']].nlargest(columns = 'median_house_value', n = 2).index[1]

display(cali_df.corr(numeric_only=True))

print(highest_corr)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924616,-0.109357,0.045480,0.069608,0.100270,0.056513,-0.015550,-0.045398
latitude,-0.924616,1.000000,0.011899,-0.036667,-0.066983,-0.108997,-0.071774,-0.079626,-0.144638
housing_median_age,-0.109357,0.011899,1.000000,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278,0.106432
total_rooms,0.045480,-0.036667,-0.360628,1.000000,0.930380,0.857281,0.918992,0.197882,0.133294
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723,0.049686
population,0.100270,-0.108997,-0.295787,0.857281,0.877747,1.000000,0.907186,0.005087,-0.025300
households,0.056513,-0.071774,-0.302768,0.918992,0.979728,0.907186,1.000000,0.013434,0.064894
median_income,-0.015550,-0.079626,-0.118278,0.197882,-0.007723,0.005087,0.013434,1.000000,0.688355
median_house_value,-0.045398,-0.144638,0.106432,0.133294,0.049686,-0.025300,0.064894,0.688355,1.000000


median_income


##### Step 5: Model 1 Linear Regression -  Try the Simple Model to analyze as first model comparision (to baseline)


In [431]:
#Simple linear regression model (no pipeline utilized for this model for now)
# Keep the feature with the best correlation from the correlation matrix exercise above which was median_income 
model_1_train_mse = ''
model_1_test_mse = ''

### BEGIN SOLUTION
X1 = X_train[['median_income']]

lr = LinearRegression().fit(X1, y_train)
model_1_train_mse = mean_squared_error(y_train, lr.predict(X1))
model_1_test_mse = mean_squared_error(y_test, lr.predict(X_test[['median_income']]))
### END SOLUTION

# Answer check
print(f'Train MSE: {model_1_train_mse: .2f}')
print(f'Test MSE: {model_1_test_mse: .2f}')

Train MSE:  7057254233.83
Test MSE:  6830312054.12


##### Step 6: Examine the VIF or variance of the correlation matrix per Savio's instruction / example in Colab Notebook as an additional indicator for feature engineering
*Questions: Will there be great colinearity with the variable (categorical) that are / is encoded?  This may be a reason to remove the feature before the VIF and a question to get further clarification on for 8.1 Try It*  

In [432]:
# VIF - measure of the multicollinearity in the independent features from learning instructor function provided: 

def vif(exogs, data):
  vif_dict = {}

  for exog in exogs:
    not_exog = [i for i in exogs if i != exog]
    X, y = data[not_exog], data[exog]

    r_squared = LinearRegression().fit(X,y).score(X,y)

    # calc the VIF
    vif = 1/(1-r_squared)
    vif_dict[exog] = vif

  return pd.DataFrame({"VIF":vif_dict})



In [433]:
#For multicolinerarty VIF analysis , drop the Categorical feature Ocean_Proximity - given it has 4 values to encode, getting around the divide by 0 is a question and if there is value here so dropping it for this model

X2 = X.drop(columns =['ocean_proximity'])
vif(X2.columns, X2).sort_values(by = 'VIF', ascending = False)

,VIF
total_bedrooms,36.003726
households,35.136045
total_rooms,12.717000
latitude,8.828919
longitude,8.713740
population,6.371238
median_income,1.731511
housing_median_age,1.260015


In [434]:
# Drop the following features based upon multicolinearity outcome for highest VIF values for total_bedrooms and households features and the Object column of ocean_proximity
X2 = X.drop(columns =['total_bedrooms', 'households', 'ocean_proximity'])
vif(X2.columns, X2).sort_values(by = 'VIF', ascending = False)


,VIF
latitude,8.316903
longitude,8.197084
total_rooms,4.725402
population,4.479289
median_income,1.308668
housing_median_age,1.257530


##### Step 7: Model 2 Linear regression accounting for multicolinearity impact and VIF analysis (remove features: total_bedrooms, households) and remove categorical feature ocean_proximity)
*Use the X2 dataframe* 

In [435]:
#Create a new train and test data set based upon the X2 dataframe considering outcome of VIF calculation and feature removal 
# Model 3 linear regression model to assess MSE for this model 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.2, random_state = 38)

lr2 = LinearRegression().fit(X2_train, y2_train)

model_2_train_mse = mean_squared_error(y2_train, lr2.predict(X2_train))
model_2_test_mse = mean_squared_error(y2_test, lr2.predict(X2_test))

print(f'Test MSE: {model_2_train_mse: .2f}')
print(f'Test MSE: {model_2_test_mse: .2f}')



Test MSE:  5219426089.97
Test MSE:  5026726177.95


##### Permutation Importance Assessment on Model 2
*Before categorical feature was encoded and incorporated into the model , linreg in this case so no consideration for ocean_proximity feature*

In [436]:
r = permutation_importance(lr2, X2_test, y2_test, random_state = 38)
r.importances_mean

array([1.42139048, 1.61428171, 0.03018604, 0.18654204, 0.1408071 ,
       0.62330448])

In [437]:
X2.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income'],
      dtype='object')

In [438]:
for i in zip(X2.columns, r.importances_mean):
  print(i)

('longitude', 1.4213904839967324)
('latitude', 1.614281709118248)
('housing_median_age', 0.030186036540214168)
('total_rooms', 0.18654204396396215)
('population', 0.14080709505372194)
('median_income', 0.6233044823945294)


##### Step 8: Model 3 - Linear Regression with OHE Feature (ocean_proximity) added and VHF features removed  

*Create make_column_transformer instance for use in pipeline with one hot encoding of ocean_proximity categorical feature to add to model 3*

In [439]:
col_transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['ocean_proximity']), 
                                          remainder='passthrough')


In [440]:
col_transformer.fit_transform(X_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population','ocean_proximity']])

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 1.600e+01, 1.593e+03,
        8.360e+02],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 3.200e+01, 1.078e+03,
        5.550e+02],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 2.400e+01, 5.372e+03,
        3.002e+03],
       ...,
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 2.500e+01, 4.096e+03,
        2.128e+03],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 3.700e+01, 2.990e+02,
        3.180e+02],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 9.000e+00, 4.855e+03,
        2.098e+03]])

*Use Pipeline to create the Model 3 to add OHE feature to linear regression model*

In [441]:
pipe_1 = ''

pipe_1 = Pipeline([('col_transformer', col_transformer), ('linreg', LinearRegression())])
pipe_1.fit(X_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income', 'ocean_proximity']], y_train)


pred_train = pipe_1.predict(X_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income', 'ocean_proximity']])
pred_test = pipe_1.predict(X_test[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income', 'ocean_proximity']])

pipe_1_train_mse = mean_squared_error(y_train, pred_train)
pipe_1_test_mse = mean_squared_error(y_test, pred_test)

print(pipe_1.named_steps)
print(f'Train MSE: {pipe_1_train_mse: .2f}')
print(f'Test MSE: {pipe_1_test_mse: .2f}')
pipe_1


{'col_transformer': ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='if_binary'),
                                 ['ocean_proximity'])]), 'linreg': LinearRegression()}
Train MSE:  5032251654.77
Test MSE:  4863596979.96


Pipeline(steps=[('col_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['ocean_proximity'])])),
                ('linreg', LinearRegression())])

##### Execute Permutation Importance calculation on Model 3
*Consider executing permutation Importance after categorical feature is considered (encoded) and incorporated into the model - in this case linreg model*

In [452]:
r1 = permutation_importance(pipe_1, X_test, y_test, random_state = 38)
r1.importances_mean

array([0.50029139, 0.52135102, 0.02441429, 0.19346895, 0.        ,
       0.14516878, 0.        , 0.59715866, 0.07299993])

In [454]:
X.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [457]:
for i in zip(X.columns, r1.importances_mean):
  print(i)

('longitude', 0.5002913880775941)
('latitude', 0.521351022123692)
('housing_median_age', 0.02441428677640236)
('total_rooms', 0.19346894754948096)
('total_bedrooms', 0.0)
('population', 0.14516877826995653)
('households', 0.0)
('median_income', 0.5971586567886835)
('ocean_proximity', 0.0729999266513844)


##### Step 9:  Model 4 - Polynomial Feature with highest correlation based upon initial correlation matrix & the VIF assessment and 

In [473]:
poly_ohe = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['ocean_proximity']),
                                           (PolynomialFeatures(include_bias = False, degree = 2), ['median_income']))
pipe_2 = Pipeline([('transformer', poly_ohe), 
                  ('linreg', LinearRegression())])

In [474]:
pipe_2.fit(X_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
      'population', 'median_income', 'ocean_proximity']], y_train)


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['ocean_proximity']),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures(include_bias=False),
                                                  ['median_income'])])),
                ('linreg', LinearRegression())])

In [475]:
quad_train_mse = ''
quad_test_mse = ''

quad_train_preds = pipe_2.predict(X_train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
      'population', 'median_income', 'ocean_proximity']])
quad_test_preds = pipe_2.predict(X_test[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
     'population', 'median_income', 'ocean_proximity']])
quad_train_mse = mean_squared_error(y_train, quad_train_preds)
quad_test_mse = mean_squared_error(y_test, quad_test_preds)

print(f'Train MSE: {quad_train_mse: .2f}')
print(f'Test MSE: {quad_test_mse: .2f}')


Train MSE:  5488558341.70
Test MSE:  5389136197.04


##### Step 9: Model 5 - Linear Regression Add all features and polynomial expansion into the final model to include categorical OHE feature (ocean_proximity) 
*Model considerations ionclude:  VHF analysis, Permutation Importance feedback* 
*Dropped features are household and total_rooms*
*2 executions of Permutation Importance shows differences but will keep these features for this iteration and expand further to consider additional models to optimize Final with Model 5 MSE*

In [445]:
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income', 'ocean_proximity']

In [446]:
X_train[features].head()


,longitude,latitude,housing_median_age,total_rooms,population,median_income,ocean_proximity
20310,-119.14,34.17,16.0,1593.0,836.0,2.7260,NEAR OCEAN
1009,-121.76,37.68,32.0,1078.0,555.0,3.1856,INLAND
9506,-123.34,39.10,24.0,5372.0,3002.0,3.0652,<1H OCEAN
105,-122.24,37.82,52.0,1509.0,674.0,4.9306,NEAR BAY
1701,-122.30,37.97,30.0,4030.0,1777.0,3.6393,NEAR BAY


In [447]:
poly_all_ohe = make_column_transformer((PolynomialFeatures(), make_column_selector(dtype_include=np.number)),
                                               (OneHotEncoder(drop = 'if_binary', sparse = False), ['ocean_proximity']))

In [448]:
train_mses = []
test_mses = []
#for degree in 1 - 5
for i in range(1, 6):
    #create pipeline with PolynomialFeatures degree i
    poly_ordinal_ohe = make_column_transformer((PolynomialFeatures(degree = i), make_column_selector(dtype_include=np.number)),
                                               (OneHotEncoder(drop = 'if_binary'), ['ocean_proximity']))
    pipe = Pipeline([('transformer', poly_ordinal_ohe), ('linreg', LinearRegression())])

    pipe.fit(X_train[features], y_train)
    #fit on train
    p1 = pipe.predict(X_train[features])
    p2 = pipe.predict(X_test[features])
    #predict on train and test
    train_mses.append(mean_squared_error(y_train, p1))
    test_mses.append(mean_squared_error(y_test, p2))
### END SOLUTION

# Answer check
print(train_mses)
print(test_mses)
pipe

[5032251654.774125, 4469624694.512497, 4043333669.374456, 3910522821.1459723, 3970723076.1012607]
[4863596979.963902, 4406448626.121298, 8805543625.75255, 121964503406.51906, 8429941106649.87]


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('polynomialfeatures',
                                                  PolynomialFeatures(degree=5),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x28c48dd10>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['ocean_proximity'])])),
                ('linreg', LinearRegression())])

##### Output Best Model from Final Model as the Best Model 

In [449]:
best_complexity = test_mses.index(min(test_mses)) + 1
best_mse = min(test_mses)
### END SOLUTION

# Answer check
print(f'The best degree polynomial model is:  {best_complexity}')
print(f'The smallest mean squared error on the test data is : {best_mse: .2f}')

The best degree polynomial model is:  2
The smallest mean squared error on the test data is :  4406448626.12
